## Introduction

- I checked the dataset for outliers. 
- I analyzed body metrics from the dataset to find out how smoking affects organs and body functions.
- I looked at heart function, blood pressure, cholesterol, hemoglobin, liver function, eyesight and provided insights.
- I built a machine learning model using random forest and made prediction on the test set.

Note: I have used the non-competition version of the dataset and split it into train and test sets.

## Table of Contents

**[I. Data Overview](#chapter1)**

[1. Are there outliers?](#subsection1)

[2. Summary](#subsection2)

**[II. Feature Engineering](#chapter2)**

[1. What do we know about the people in the dataset?](#subsection3)

[2. Body Metrics](#subsection4)

[3. Senses: Do non-smokers have better eyesight and hearing?](#subsection5)

[4. Heart: Do smokers have a higher chance of heart diseases?](#subsection6)

[5. Cholesterol: Does smoking increase cholesterol?](#subsection7)

[6. Liver: Are liver functions worse for smokers?](#subsection8)

[7. Blood: Do smokers have higher haemoglobin?](#subsection9)

**[III. Random Forest Model](#chapter3)**

[1. Split dataset](#subsection10)

[2. Build model](#subsection11)

**[IV. Conclusion](#chapter4)**

## I. Data Overview <a class="anchor"  id="chapter1"></a>

**Libraries**

In [ ]:
library(dplyr)
library(tidyverse)
library(caret)
library(grid)
library(gridExtra)

In [ ]:
df <- read.csv("../input/body-signal-of-smoking/smoking.csv")
summary(df)
str(df)

**Transform character columns**

In [ ]:
unique(df$tartar)
unique(df$gender)
unique(df$oral)
df$sex_num <- ifelse(df$gender=="F",0,1)
df$tartar <-  ifelse(df$tartar=="Y",1,0)
df$oral <- as.numeric(as.factor(df$oral))

### 1. Are there outliers? <a class="anchor"  id="subsection1"></a>
**Let's check outliers for continuous variables**

In [ ]:
df_num<-select_if(df,is.numeric) %>% select(-c(ID,hearing.right.,hearing.left.,smoking,dental.caries,Urine.protein))
df_num_p<-df_num %>% gather(variable,values,1:18)

options(repr.plot.width = 18, repr.plot.height = 14)
ggplot(df_num_p)+
  geom_boxplot(aes(x=variable,y=values),fill="royalblue") + 
  facet_wrap(~variable,ncol=3,scales="free") + 
  theme(strip.text.x = element_blank(),
        text = element_text(size=18))

**Let's zoom in on the columns which supposedly have outliers**
- age
- eyesight left and right
- HDL
- height
- triglyceride
- ALT

In [ ]:
a <- ggplot(df, aes(x="",y=age))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  scale_y_continuous(breaks=seq(0,100,5))+
  labs(y="Age")

b <- ggplot(df, aes(x="",y=eyesight.left.))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="Eyesight left")

c <- ggplot(df, aes(x="",y=eyesight.right.))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="Eyesight right")

d <- ggplot(df, aes(x="",y=HDL))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="HDL")

e <- ggplot(df, aes(x="",y=height.cm.))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="Height")

f <- ggplot(df, aes(x="",y=triglyceride))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="Triglyceride")

g <- ggplot(df, aes(x="",y=ALT))+
  geom_boxplot(fill="steelblue", alpha=0.6)+
  theme_bw()+
  labs(y="ALT")

options(repr.plot.width = 18, repr.plot.height = 12)
grid.arrange(a,b,c,d,e,f,g, ncol=4)

**Let's get the summary of the boxplots with the count of outliers**

In [ ]:
summary(boxplot.stats(df$age))
summary(boxplot.stats(df$eyesight.left.))
summary(boxplot.stats(df$eyesight.right.))
summary(boxplot.stats(df$HDL))
summary(boxplot.stats(df$height.cm.))
summary(boxplot.stats(df$ALT))
summary(boxplot.stats(df$triglyceride))

**The summary of boxplots show that the count of outliers for all the suspected variables are > 200. They cannot be considered as outliers**

### 2. Summary <a class="anchor"  id="subsection2"></a>
- There are no blanks or NAs.
- No outliers noticed
- Convert height weight to BMI (maybe)

## II. Feature Engineering <a class="anchor"  id="chapter2"></a>

### 1. What do we know about the people in the dataset? <a class="anchor"  id="subsection3"></a>

In [ ]:
df$age_grp <- cut(df$age, c(0,17,60,100,120), labels = c("0-17","18-60","60-100","100+") )

h <- ggplot(df,aes(x=`age_grp`, fill=`gender`))+
  geom_bar()+
  facet_grid(.~gender)+
  stat_count(aes(y=..count.., label=..count..), vjust=-0.5,geom="text", col="black", size=3.5)+
  labs(x="Age Group", y = "Count", title="Age Group vs Sex", fill= "Sex")+
  theme(plot.title=element_text(face="bold",  hjust=0.5), legend.position = "bottom", legend.background = element_rect(fill="gray80"),text=element_text(size=18))+
    scale_fill_manual(values=c("plum3","royalblue"))

i <- ggplot(df, aes(y = gender,fill=gender))+
  geom_bar()+
  scale_fill_manual(values=c("plum3","royalblue"))+
  labs(title="Sex",x=" ")+
  theme(legend.position = "none", plot.title=element_text(face="bold",  hjust=0.5), text=element_text(size=18))+
  coord_polar("y")

options(repr.plot.width = 16, repr.plot.height = 12)
grid.arrange(i, h, ncol=1)

#### 1.1. Insights
- There are more females than males
- Age group 18-60 has more people among both male and female
- Everyone in the dataset is an adult

### 2. Body metrics <a class="anchor"  id="subsection4"></a>

In [ ]:
df$bmi <- (df$weight.kg./(df$height.cm.*df$height.cm.))*10000
df$bmi_cat <- ifelse(df$bmi<18.5,"Underweight", NA )
df$bmi_cat <- ifelse(df$bmi>=18.5 & df$bmi <=24.9,"Healthy weight", df$bmi_cat )
df$bmi_cat <- ifelse(df$bmi > 24.9 & df$bmi<=29.9,"Overweight", df$bmi_cat)
df$bmi_cat <- ifelse(df$bmi > 30,"Obesity", df$bmi_cat)
table(df$bmi_cat)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 12)
ggplot(df, aes(x=bmi_cat, fill= as.factor(smoking)))+
  geom_bar(position="dodge", col="black", alpha=0.9)+
  scale_fill_manual(label=c("no","yes"),values=c("cadetblue", "firebrick"))+
  theme(legend.position = "bottom")+
  labs(x=" ", fill="Smoking", title="BMI and Smoking")+
  theme(text=element_text(size=18), plot.title= element_text(face="bold", hjust=0.5))

#### 2.1. Insights
- Maximum smokers have normal weight, followed by overweight people. Underweight people have the least number of smokers. 
- To calculate BMI , I obtained the formula from CDC (https://www.cdc.gov/nccdphp/dnpao/growthcharts/training/bmiage/page5_1.html#:~:text=With%20the%20metric%20system%2C%20the,by%2010%2C000%2C%20can%20be%20used.)

### 3. Senses: Do non-smokers have better eyesight and hearing? <a class="anchor"  id="subsection5"></a>

In [ ]:
cat(paste("\n \n Mean left eyepower of smokers: ",mean(df[df$smoking==1,'eyesight.left.']), "\n \n "))
cat(paste("Mean left eyepower of non-smokers: ",mean(df[df$smoking==0,'eyesight.left.']), "\n \n "))
cat(paste("Mean right eyepower of smokers: ",mean(df[df$smoking==1,'eyesight.right.']),"\n \n "))
cat(paste("Mean right eyepower of non-smokers: ",mean(df[df$smoking==0,'eyesight.right.']), "\n \n "))

#### 3.1. Insights
**People who smoke have higher eye power than people who do not smoke.**

### 4. Heart: Do smokers have a higher chance of heart diseases? <a class="anchor"  id="subsection6"></a>

To measure the state of the heart or predict who is prone to cardiovascular diseases, systolic and diastolic (relaxed) blood pressure are commonly used metrics. 

In [ ]:
BP <- c("Normal","Elevated","Hypertension I","Hypertension II", "Hypertensive Crisis")
sys <- c("less than 120", "120-129", "130-139", "140 or higher", "more than 180")
dias <- c("less than 80", " less than 80", "80-89", "90 or higher", "120 or higher")
pressure <- data.frame(BP, sys,dias)
colnames(pressure) <- c("Blood Pressure", "Systolic", "Diastolic (Relaxed)")
grid.newpage()
grid.table(pressure)

In [ ]:
df$pressure <- ifelse(df$systolic<120 & df$relaxation<80,"Normal",NA)
df$pressure <- ifelse(df$systolic>=120 & df$relaxation <80, "Elevated",df$pressure)
df$pressure <- ifelse(df$systolic>=130 | df$relaxation>=80,"Hypertension Stage 1",df$pressure)
df$pressure <- ifelse(df$systolic>=140 | df$relaxation>=90,"Hypertension Stage 2",df$pressure)
df$pressure <- ifelse(df$systolic>=180 | df$relaxation>=120,"Hypertensive Crisis",df$pressure)
table(df$pressure)

In [ ]:
table <- df %>% 
  group_by(df$pressure)%>%
  summarise(smoker= length(smoking[smoking=="1"]), 
            nonsmoker= length(smoking[smoking=="0"]),
            smoker_percent= smoker/sum(smoker,nonsmoker)*100)%>%
  arrange(desc(smoker_percent))

grid.newpage()
grid.table(table)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 12)
ggplot(df, aes(x=reorder(pressure,pressure,function(x)-length(x)), fill=as.factor(smoking)))+
  geom_bar(col="black",position="stack")+
  theme(plot.title=element_text(face="bold",hjust=0.5),legend.position = "bottom", axis.text.x = element_text(angle=30), text=element_text(size=18))+
  labs(title="Blood Pressure and Smoking",x="Blood Pressure", fill="Smoking")+
  stat_count(aes(y=..count..,label=..count..), geom="text", vjust=1.5, size=8, col="white")+
  scale_fill_manual(label=c("no","yes"),values=c("cadetblue", "firebrick"))

#### 4.1. Insights
**As the table shows, the % of smokers within each blood pressure group is the highest among those who have hypertensive crisis and the least among those with normal blood pressure**
- Maximum people in the dataset have normal blood pressure
- The highest number of nonsmokers is in the normal blood pressure

Information source:
https://www.heart.org/en/health-topics/high-blood-pressure/understanding-blood-pressure-readings 

### 5. Does smoking increase cholesterol? <a class="anchor"  id="subsection7"></a> 
Cholesterol: Do smokers have lower good fat HDL? Does smoking increase harmful fats? 

#### 5.1 HDL (the good cholesterol)
**Hypothesis: HDL is higher for non-smokers.**

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 12)
ggplot(subset(df,df$HDL<200), aes(x=HDL, fill=as.factor(smoking)))+
  geom_bar(alpha=0.8)+
  labs(title="HDL and Smoking", fill="Smoking")+
  theme(legend.position = "bottom", plot.title = element_text(face="bold", hjust=0.5), text=element_text(size=18))+
  scale_x_continuous(breaks=seq(0,200,20))+
  scale_fill_manual(labels=c("No","Yes"),values=c("forest green","red"))+
  geom_vline(aes(xintercept=mean(df$HDL)))

**As the graph shows, my hypothesis is correct. More non-smokers have higher HDL and it is lower among smokers** 

#### 5.2. LDL (the bad cholesterol)

**Hypothesis: Average LDL is higher for smokers.**

In [ ]:
df$LDL_stat <- case_when(
    df$LDL<100 ~ "healthy",
    df$LDL>=100 ~ "unhealthy")
table(df$smoking, df$LDL_stat)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)
ggplot(subset(df, df$LDL<400), aes(x=LDL_stat, fill=as.factor(bmi_cat)))+
  geom_bar(col="black")+
  labs(title="LDL and Smoking", fill="BMI group")+
  theme(legend.position = "bottom", plot.title = element_text(face="bold", hjust=0.5), text=element_text(size=18))+
  facet_grid(.~smoking)+
  scale_fill_brewer(palette="Set2")

cat(paste("\n \n Average LDL of female smokers: ",mean(df[df$gender=="F" & df$smoking==1, 'LDL'])))

cat(paste("\n \n Average LDL of female non-smokers: ",mean(df[df$gender=="F" & df$smoking==0, 'LDL'])))

cat(paste("\n \n Average LDL of male smokers: ",mean(df[df$gender=="M" & df$smoking==1, 'LDL'])))

cat(paste("\n \n Average LDL of male non-smokers: ",mean(df[df$gender=="M" & df$smoking==0, 'LDL']), "\n \n \n"))

**The hypothesis is false. Contrarily, the chart shows that non-smokers have a significantly higher LDL level than smokers. Average LDL levels are higher for both men and women who do NOT smoke.** 

#### 5.3. Total cholesterol

In [ ]:
df$chol_grp <- ifelse(df$Cholesterol>=125 & df$Cholesterol<=200,"Healthy", "Risk")
options(repr.plot.width = 12, repr.plot.height = 10)
ggplot(df,aes(x=as.factor(smoking), fill=as.factor(chol_grp)))+
  geom_bar(col="black",position="dodge", alpha=0.8)+
  scale_fill_manual(values=c("dark seagreen","coral"))+
  labs(title="Cholesterol and Smoking", fill="Smoking", x="Cholesterol Level")+
  geom_text(aes(label=..count..), stat="count", position= position_dodge(width=1), vjust=1.5, size=6)+
  theme(legend.position = "bottom", plot.title = element_text(face="bold", hjust=0.5), text=element_text(size=18))

**In the non-smoker group, more people have healthy cholesterol. However, in the smoker group, more people also have healthy cholesterol.**

#### 5.4. Triglycerides

The cutoff for triglyceride is 150, above 150 is at risk. 

In [ ]:
df%>% select(c("triglyceride", "smoking")) %>% 
  group_by(smoking) %>%
  summarise(tryg_mean = mean(triglyceride)) %>%
  mutate(., Type=ifelse(tryg_mean<150,"Healthy","Risk"))

**The average triglyceride for smokers is at risk while that of non-smokers is healthy**

#### 5.5. Insights
- No strong indication can be seen where smoking and cholesterol share a causal relation. However, it cannot be ruled out either and there can be a correlation. 

Information source:
https://medlineplus.gov/cholesterollevelswhatyouneedtoknow.html 

### 6. Liver: Are liver functions worse for smokers? <a class="anchor"  id="subsection8"></a>
- AST normal range -  8 to 33 U/L 
- ALT normal range - 4 to 36 U/L
- GTP normal range - 5 to 40 U/L  

In [ ]:
df%>% select(c("ALT", "AST", "Gtp", "smoking")) %>% 
  group_by(smoking) %>%
  summarise(Alt_mean = mean(ALT),
            Ast_mean = mean(AST),
            Gtp_mean = mean(Gtp)) 

#### 6.1. Insights
- AST and ALT are healthy for both smoker and non-smoker groups. GTP is at risk for smokers. 

### 7. Blood: Do smokers have higher haemoglobin? <a class="anchor"  id="subsection9"></a>

In [ ]:
mean(df[df$smoking=="1",'hemoglobin'])
mean(df[df$smoking=="0",'hemoglobin'])

options(repr.plot.width = 14, repr.plot.height = 10)
ggplot(df, aes(y=hemoglobin))+
  geom_boxplot(fill=c("cadetblue","firebrick2"))+
  facet_grid(.~as.factor(smoking))+
  scale_y_continuous(breaks= seq(0,20,2))+
  labs(title="Hemoglobin and Smoking", fill="smoking")+
  theme(plot.title = element_text(face="bold", hjust=0.5), text=element_text(size=18))

#### 7.1. Insights
- The mean hemoglobin for smokers is slightly higher (15.4) that of non-smokers (14.1)

## III. Random Forest Model <a class="anchor"  id="chapter3"></a>

### 1. Split dataset <a class="anchor"  id="subsection10"></a>

In [ ]:
# Split data
library(caTools)
set.seed(123)
df$split<- sample.split(df$smoking, SplitRatio = 0.7)
df_train <- df %>% filter( split==TRUE) %>% select(-split)
df_test <- df%>% filter(split==FALSE)%>%select(-split)
 
paste0("Number of rows in train dataset: ",nrow(df_train))
paste0("Number of rows in test dataset: ", nrow(df_test))

### 2. Build model <a class="anchor"  id="subsection10"></a>

In [ ]:
library(randomForest)
set.seed(1234)

model1 <- randomForest(factor(smoking) ~ age + height.cm. + weight.kg.+ sex_num + 
                         eyesight.left. + eyesight.right. + hearing.left. + hearing.right. +
                         systolic + relaxation + 
                         Cholesterol + triglyceride + HDL + LDL +
                         fasting.blood.sugar + hemoglobin + Urine.protein + serum.creatinine+
                         AST + ALT + Gtp +
                         dental.caries + tartar,
                                importance=TRUE,
                                ntree= 500,
                                data = df_train)
paste("done")

#### Model Error

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 10)
plot(model1, ylim=c(0,0.36),main="Error Plot")
legend('topright', colnames(model1$err.rate), col=1:3, fill=1:3)

#### Variable Importance

In [ ]:
imp <- importance(model1)
options(repr.plot.width = 14, repr.plot.height = 10)
varImpPlot(model1, main="Variable Importance")

# Creating a dataframe of variable importance metrics
df_imp<- data.frame(variable=row.names(imp),
                     importance = imp[,'MeanDecreaseGini'],
                     accuracy= imp[,'MeanDecreaseAccuracy'])
noquote("Variable Importance Dataframe")
df_imp
# Visualizations of importance variables
options(repr.plot.width = 14, repr.plot.height = 10)
ggplot(df_imp, aes(x=reorder(variable,-importance),y=importance, fill=importance))+
  geom_col()+
  labs(x="Predictors", y ="Importance", title="Importance of predictor variables")+
  theme(plot.title=element_text(face="bold",hjust=0.5), legend.position = "bottom", axis.text.x = element_text(angle=45), text=element_text(size=18))+
  scale_fill_continuous(type="viridis")

**Let's remove some of the low-contribution variables and improve the model**

In [ ]:
model2 <- randomForest(factor(smoking) ~ age + height.cm. + weight.kg.+ sex_num + 
                         systolic + relaxation + 
                         Cholesterol + triglyceride + HDL + LDL +
                         fasting.blood.sugar + hemoglobin + serum.creatinine+
                         AST + ALT + Gtp +
                         dental.caries ,
                                importance=TRUE,
                                ntree= 500,
                                data = df_train)
paste("done")
model2

In [ ]:
model1

cat(paste("Accuracy of model 1: ", (100-17.44), "%"))

cat(paste("Accuracy of model 2: ", (100-17.76),"%"))

### 3. Prediction and Submission
Model 1 has higher accuracy than model 2. I'll go with model 1 for prediction.

In [ ]:
#library(caret)
pred <- predict(model1, newdata =  df_test)
result <- data.frame(id= df_test$ID ,smoking = df_test$smoking)
result[1:20,]

## IV. Conclusion <a class="anchor"  id="chapter4"></a>
Smoking is injurious to health. Here are some observations suppporting this statement:
- Higher eyepower has been observed among smokers
- Hyperintensive and high blood pressure tend to be correlated with smokers. This can cause heart diseases.
- The good cholesterol is seen less in smokers.
- Triglyceride is seen at a risk level among smokers
- Gtp, a metric of liver function is seen to be at a risk level among smokers

#### _Thank you for reading!_